In [12]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [17]:
team_stats1415 = pd.read_csv("./team_stats1415.csv", sep = ",")
team_stats1516 = pd.read_csv("./team_stats1516.csv", sep = ",")
team_stats1617 = pd.read_csv("./team_stats1617.csv", sep = ",")
team_stats1718 = pd.read_csv("./team_stats1718.csv", sep = ",")
scraped_team_stats1718 = pd.read_csv("./scraped_EOS_2017-18.csv", sep = ",")

In [27]:
team_stats1718

,Unnamed: 0,TEAM,GP,W,L,WIN%,PTS,FGM,FGA,FG%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PLUS/MINUS
0,0,Houston_Rockets,82,65,17,0.793,112.4,38.7,84.2,46.0,...,34.5,43.5,21.5,13.8,8.5,4.8,4.4,19.5,20.4,8.5
1,1,Toronto_Raptors,82,59,23,0.720,111.7,41.3,87.4,47.2,...,34.2,44.0,24.3,13.4,7.6,6.1,4.9,21.7,19.9,7.8
2,2,Golden_State_Warriors,82,58,24,0.707,113.5,42.8,85.1,50.3,...,35.1,43.5,29.3,15.4,8.0,7.5,3.7,19.6,18.5,6.0
3,3,Boston_Celtics,82,55,27,0.671,104.0,38.3,85.1,45.0,...,35.1,44.5,22.5,14.0,7.4,4.5,4.4,19.7,19.2,3.6
4,4,Philadelphia_76ers,82,52,30,0.634,109.8,40.8,86.6,47.2,...,36.5,47.4,27.1,16.5,8.3,5.1,5.1,22.1,20.4,4.5
5,5,Cleveland_Cavaliers,82,50,32,0.610,110.9,40.4,84.8,47.6,...,33.7,42.1,23.4,13.7,7.1,3.8,4.1,18.6,20.7,0.9
6,6,Portland_Trail_Blazers,82,49,33,0.598,105.6,39.3,87.0,45.2,...,35.3,45.5,19.5,13.5,7.0,5.2,5.1,19.5,19.1,2.6
7,7,Indiana_Pacers,82,48,34,0.585,105.6,40.8,86.4,47.2,...,32.7,42.3,22.2,13.3,8.8,4.1,5.2,18.8,18.5,1.4
8,8,Oklahoma_City_Thunder,82,48,34,0.585,107.9,39.9,88.1,45.3,...,32.6,45.1,21.3,14.0,9.1,5.0,4.6,20.2,21.3,3.4
9,9,New_Orleans_Pelicans,82,48,34,0.585,111.7,42.7,88.3,48.3,...,35.7,44.3,26.8,14.9,8.0,5.9,4.2,19.1,20.4,1.3
